In [11]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/standings?standingstype=division&season=46&conference=8&division=-1&context=overall&specialteams=false&sortkey=points&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for row in rows[1:]:  # Skip header row
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Team', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PCT', 'PCT2', 'PIM', 'RW', 'ROW', 'STK', 'IN-DIV', 'GPCT', 'P10', 'Nickname']


data_without_first_two = [row[2:] for row in acha_dynamic_stats]

df = pd.DataFrame(data_without_first_two, columns=column_names)


int_columns = ['RW', 'ROW', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PIM']
for column in int_columns:
    df[column] = pd.to_numeric(df[column]).fillna(0).astype(int)


float_columns = ['PCT', 'GPCT', 'PCT2']
for column in float_columns:
    df[column] = pd.to_numeric(df[column])


string_columns = ['Team', 'STK', 'IN-DIV', 'P10', 'Nickname']
for column in string_columns:
    df[column] = df[column].astype(str)


df = df[df['Team'] != 'None']
df = df.dropna(subset=['Team'])

print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_md1_team.csv', index=False)

     Team  GP   W   L  OTL  T  SOW  SOL  PTS  OTW  ...    PCT  PCT2  PIM  RW  \
0   M1JCU  21  16   4    1  0    1    0   33    1  ...  0.786   0.0  228  14   
1   M1DUQ  21  10   9    1  0    1    1   22    0  ...  0.524   0.0  384   9   
2   M1IUP  16  10   6    0  0    2    0   20    0  ...  0.625   0.0  463   8   
3   M1KSU  19   9   9    1  0    0    0   19    1  ...  0.500   0.0  227   8   
4   M1WVU  20   8  10    1  0    0    1   18    1  ...  0.450   0.0  251   7   
..    ...  ..  ..  ..  ... ..  ...  ...  ...  ...  ...    ...   ...  ...  ..   
77  M1UCO  18   8   9    1  0    0    0   17    1  ...  0.472   0.0  292   7   
78   M1AZ  17   8   9    0  0    1    0   16    1  ...  0.471   0.0  340   6   
79  M1CSU  18   7   9    2  0    1    0   16    0  ...  0.444   0.0  224   6   
80  M1ASU  17   4  11    1  0    0    1   10    0  ...  0.294   0.0  273   4   
81  M1SDS  20   4  15    0  0    0    1    9    0  ...  0.225   0.0  561   4   

    ROW      STK   IN-DIV   GPCT      P